In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold, RandomizedSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer, precision_score, f1_score, RocCurveDisplay, roc_auc_score, fbeta_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from typing import Tuple
from collections import defaultdict

# Load and merge CSVs

In [27]:
# Function to merge CSV files
def merge_csvs(csv_list):
    # Load the first CSV
    merged_df = pd.read_csv(csv_list[0], index_col=0)
    
    # Merge the remaining CSVs
    for csv in csv_list[1:]:
        temp_df = pd.read_csv(csv)
        merged_df = merged_df.merge(temp_df, on="ID")
    
    return merged_df

# Train and compare models

### 1. Split dataset

A common pitfalls when evaluating and testing a classification model for potential lung nodule malignancy is not considering the nodules of the same patient to be completely separated into the training set or the validation set [1].

In [3]:
def split_dataset(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    # Step 1: Get unique patient_ids
    unique_patients = df['Patient_ID'].unique()

    # Step 2: Split unique patient_ids
    train_patients, test_patients = train_test_split(unique_patients, test_size=0.2, random_state=42)

    # Step 3: Filter original dataset based on the split patient_ids
    train_set = df[df['Patient_ID'].isin(train_patients)]
    test_set = df[df['Patient_ID'].isin(test_patients)]
    return train_set, test_set

In [4]:
def prepare_df(merged_df):
    merged_df = merged_df[merged_df["Malignancy"] != 3] # Remove Indeterminate malignancy
    merged_df.loc[merged_df["Malignancy"] < 3, "Malignancy"] = 0
    merged_df.loc[merged_df["Malignancy"] > 3, "Malignancy"] = 1

    train_set, test_set = split_dataset(merged_df)

    # Define features and target
    X_train = train_set.drop(columns=["ID", "Scan_ID", "Patient_ID", "Malignancy"])  # Exclude "ID", "Scan_ID", "Patient_ID" and "Malignancy" from features
    y_train = train_set["Malignancy"]  # Target variable
    groups_train = train_set["Patient_ID"]

    X_test = test_set.drop(columns=["ID", "Scan_ID", "Patient_ID", "Malignancy"])  # Exclude "ID", "Scan_ID", "Patient_ID" and "Malignancy" from features
    y_test = test_set["Malignancy"]  # Target variable

    return X_train, y_train, groups_train, X_test, y_test

### 2. Hyperparameter Tuning

We used the **RandomizedSearchCV** method, that allows for randomized sampling of hyperparameters to optimize the machine learning method.
The function accepts a classifier, a set of hyperparameter distributions to search over, the input data X and labels y, and a name for the model.

> **Preprocessing:** We created a **ColumnTransformer**, which applies **StandardScaler()** to scale the numeric features. This is crucial, especially for algorithms sensitive to feature scaling like XGBoost.

> **Pipeline Setup:** We created a pipeline that consists of the preprocessing step and the classifying step.

> **Cross-Validation:** We used **StratifiedKFold** cross-validation, which splits the data into 5 folds while ensuring that the distribution of classes remains balanced in each fold.

> **RandomizedSearchCV:** The next step is to perform hyperparameter tuning by randomly selecting hyperparameter combinations from the provided **param_distributions**.

> **Fitting the Model and Displaying Results:** After the tuning process and the fitting of each model, the function prints the best hyperparameters and the highest AUC score achieved during cross-validation.

In [28]:
# Function to perform RandomizedSearchCV
def tune_model(classifier, param_distributions, X, y, groups, model_name):
    # Define the preprocessor (if needed)
    # Here, we"ll standardize the numeric features
    numeric_features = X.select_dtypes(include=["float64", "int64"]).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features)  # Standardize numeric features
        ]
    )

    # Create the pipeline
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),  # Preprocessing step
        ("classifier", classifier)  # Classifier passed as parameter
    ])

    # Setup StratifiedKFold cross-validation
    kf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

    # Setup RandomizedSearchCV
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions,
        n_iter=150,  # Number of random combinations to try
        scoring="recall",  # Use recall as the scoring metric
        cv=kf,
        random_state=42,
        n_jobs=-1,  # Use all available cores
        return_train_score=True,
        # error_score='raise',
    )

    # Perform RandomizedSearchCV
    random_search.fit(X, y, groups=groups)

    # Display the best parameters and score
    print(f"{model_name} - Best parameters: {random_search.best_params_}")
    print(f"{model_name} - Best Recall score: {random_search.best_score_:.4f}")

    return random_search.best_estimator_, random_search.best_params_

In [6]:
def plot_metrics(models, X_test, y_test):
    _, ax = plt.subplots()
    for name, pipeline in models:
        RocCurveDisplay.from_estimator(pipeline, X_test, y_test, ax=ax, name=name)
    plt.title("ROC Curve")
    plt.show()

    metrics = defaultdict(list)
    names = []
    for name, pipeline in models:
        names.append(name)
        preds = pipeline.predict(X_test)

        metrics["precision"].append(precision_score(y_test, preds))
        metrics["f1_scores"].append(f1_score(y_test, preds))
        metrics["f2_scores"].append(fbeta_score(y_test, preds, beta=2))
        metrics["balanced_accuracy"].append(balanced_accuracy_score(y_test, preds))
        metrics["aucs"].append(roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))

    plt.figure(figsize=(15, 10))
    for i, column in enumerate(metrics.keys()):
        plt.subplot(2, 3, i + 1)
        plt.bar(names, metrics[column])
        plt.ylabel("scale")
        plt.title(column)
        plt.ylim(bottom=0.65)
    plt.show()

    plt.figure(figsize=(12, 5))
    for i, (name, pipeline) in enumerate(models):
        ax = plt.subplot(2, 3, i + 1)
        plt.title(name)
        ConfusionMatrixDisplay.from_estimator(pipeline, X_test, y_test, ax=ax)
    plt.plot()

###  XGBoost (xgb_param_distributions): 


- **n_estimators:** Number of boosting rounds or trees to grow, with values ranging from 50 to 300 in steps of 50.
- **max_depth:** Maximum depth of each tree, which controls the complexity of the model, sampled between 3 and 15.
- **learning_rate:** The step size at each iteration, ranging from 0.01 to 0.2 to control the contribution of each tree.
- **subsample:** Fraction of samples to be used in each boosting round, with values of 0.5, 0.7, and 1.0.
- **colsample_bytree:** Fraction of features used when building each tree, with values of 0.5, 0.7, and 1.0.
- **gamma:** Minimum loss reduction required to make a further partition in a leaf node, ranging from 0 to 5 in steps of 0.5.

In [30]:
# Parameter distributions for each classifier
xgb_param_distributions = {
    "classifier__n_estimators": np.arange(50, 300, 50),
    "classifier__max_depth": np.arange(3, 15),
    "classifier__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "classifier__subsample": [0.5, 0.7, 1.0],
    "classifier__colsample_bytree": [0.5, 0.7, 1.0],
    "classifier__gamma": np.arange(0, 5, 0.5),
}

### Random Forest (rf_param_distributions):

- **n_estimators:** Number of trees in the forest, sampled between 50 and 300 in steps of 50.
- **max_depth:** Maximum depth of each tree, sampled between 3 and 15.
- **min_samples_split:** Minimum number of samples required to split an internal node, with options of 2, 5, and 10.
- **min_samples_leaf:** Minimum number of samples required to be at a leaf node, with values of 1, 2, and 4.
- **max_features:** The number of features to consider when looking for the best split, with options "auto" and "sqrt".

In [ ]:
rf_param_distributions = {
    "classifier__n_estimators": np.arange(50, 300, 50),
    "classifier__max_depth": np.arange(3, 15),
    "classifier__min_samples_split": [2, 5, 10],
    "classifier__min_samples_leaf": [1, 2, 4],
    "classifier__max_features": ["log2", "sqrt"],
}


''' 
3 é irrelevante , remove
1ª-> 1+2 bom, 4+5 mau                 corta-se max(1+2, 4+5)-min(1+2,4+5) a max(1+2,4+5)
2ª-> 1*0.5+2*0.5 bom, 4*0.5+5*0.5 mau
3ª-> w1, w2, w4, w5 
4ª-> >=3 mau, else bom

  ver diferênças entre as diff formas


sendo 1, 5 os resultados mais seguros dar mais importância q o 2, 4
r => nº de nodulos malig/benig
'''

In [ ]:
merged_df = merge_csvs(["annotations_ds.csv"])
X_train, y_train, groups_train, X_test, y_test = prepare_df(merged_df)

In [ ]:
# Tune models
best_xgb_model, best_xgb_params = tune_model(XGBClassifier(eval_metric="mlogloss"), xgb_param_distributions, X_train, y_train, groups_train, "XGBoost")
best_rf_model, best_rf_params = tune_model(RandomForestClassifier(random_state=42), rf_param_distributions, X_train, y_train, groups_train, "Random Forest")

In [ ]:
models = [
    ("XgBoost", best_xgb_model),
    ("Random Forest", best_rf_model),
]
plot_metrics(models, X_test, y_test)

# Annotations + Radiomics

In [17]:
merged_df = merge_csvs(["annotations_ds.csv", "radiomics.csv"])
X_train, y_train, groups_train, X_test, y_test = prepare_df(merged_df)

In [18]:
# Tune models
best_xgb_model, best_xgb_params = tune_model(XGBClassifier(eval_metric="mlogloss"), xgb_param_distributions, X_train, y_train, groups_train, "XGBoost")
best_rf_model, best_rf_params = tune_model(RandomForestClassifier(random_state=42), rf_param_distributions, X_train, y_train, groups_train, "Random Forest")

XGBoost - Best parameters: {'classifier__subsample': 0.7, 'classifier__n_estimators': 200, 'classifier__max_depth': 10, 'classifier__learning_rate': 0.1, 'classifier__gamma': 2.0, 'classifier__colsample_bytree': 0.7}
XGBoost - Best Recall score: 0.8196


/home/psuper/Documents/Escola/Universidade/3º Ano/1_sem/lab_ia/trab1/lung-cancer-classification/.venv/lib/python3.10/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Random Forest - Best parameters: {'classifier__n_estimators': 100, 'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 1, 'classifier__max_features': 'log2', 'classifier__max_depth': 12}
Random Forest - Best Recall score: 0.7928


In [ ]:
models = [
    ("XgBoost", best_xgb_model),
    ("Random Forest", best_rf_model),
]
plot_metrics(models, X_test, y_test)

## References

- [1] Causey, J.L., Zhang, J., Ma, S., Jiang, B., Qualls, J.A., Politte, D.G., Prior, F., Zhang, S. and Huang, X. (2018). Highly accurate model for prediction of lung nodule malignancy with CT scans. Scientific Reports, 8(1). doi:https://doi.org/10.1038/s41598-018-27569-w.